In [2]:
import pandas as pd

In [2]:
content_req_1 = pd.read_csv("../data/guttmacher_content_requirements_sex_ed_hiv.csv", index_col=0)

In [3]:
content_req_2 = pd.read_csv("../data/guttmacher_content_requirements_sex_ed_hiv_part_2.csv", index_col=0)

In [4]:
life_skills = pd.read_csv("../data/guttmacher_sex_ed_life_skills.csv", index_col=0)

In [5]:
flags_df = content_req_1.join(content_req_2, how="outer")

In [6]:
flags_df = flags_df.join(life_skills, how="outer")

In [7]:
flags_df

,SEX EDUCATION MANDATED,HIV EDUCATION MANDATED,Be Medically Accurate,Be Age Appropriate,Be Culturally Appropriate and Unbiased,Cannot Promote Religion,Notice: Parental Role,Consent: Parental Role,Opt-out allowed: Parental Role,Contraception: Sex Ed,...,Importance of sex Only within marriage: Sex Ed,Sexual orientation: Sex Ed,Negative outcomes of teen sex: Sex Ed,Condoms: HIV,Abstinence: HIV,Healthy relationships,Sexual decision-making and self-discipline,Refusal skills and personal boundaries,Consent,Dating and sexual violence prevention
Alabama,NaN,X,NaN,X,NaN,NaN,NaN,NaN,X,X,...,X,NaN,NaN,X,Stress,NaN,X,X,NaN,X
Alaska,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
Arizona,NaN,NaN,HIV,X,NaN,NaN,X,Sex,HIV,NaN,...,NaN,NaN,X,NaN,Stress,X,X,X,NaN,X
Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,NaN,NaN,NaN,Stress,X,X,X,NaN,X
California,X,X,X,X,X,X,X,NaN,X,X,...,NaN,Inclusive,NaN,X,Cover,X,X,X,NaN,X
Colorado,NaN,NaN,X,X,X,X,X,NaN,X,X,...,NaN,Inclusive,X,X,Cover,X,X,X,X,X
Connecticut,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,X,X,...,NaN,Inclusive,X,NaN,NaN,X,X,NaN,NaN,X
Delaware,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,...,NaN,Inclusive,NaN,X,Stress,X,X,X,X,X
Dist. of Columbia,X,X,NaN,X,NaN,NaN,X,NaN,X,X,...,NaN,Inclusive,X,NaN,Cover,X,X,X,X,X
Florida,X,X,NaN,X,NaN,NaN,NaN,NaN,X,NaN,...,X,Negative,X,NaN,Stress,X,X,NaN,NaN,X


In [8]:
flags_df = flags_df.notnull().astype('int')

In [9]:
flags_df

,SEX EDUCATION MANDATED,HIV EDUCATION MANDATED,Be Medically Accurate,Be Age Appropriate,Be Culturally Appropriate and Unbiased,Cannot Promote Religion,Notice: Parental Role,Consent: Parental Role,Opt-out allowed: Parental Role,Contraception: Sex Ed,...,Importance of sex Only within marriage: Sex Ed,Sexual orientation: Sex Ed,Negative outcomes of teen sex: Sex Ed,Condoms: HIV,Abstinence: HIV,Healthy relationships,Sexual decision-making and self-discipline,Refusal skills and personal boundaries,Consent,Dating and sexual violence prevention
Alabama,0,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,1
Alaska,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
Arizona,0,0,1,1,0,0,1,1,1,0,...,0,0,1,0,1,1,1,1,0,1
Arkansas,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,0,1
California,1,1,1,1,1,1,1,0,1,1,...,0,1,0,1,1,1,1,1,0,1
Colorado,0,0,1,1,1,1,1,0,1,1,...,0,1,1,1,1,1,1,1,1,1
Connecticut,0,1,0,0,0,0,0,0,1,1,...,0,1,1,0,0,1,1,0,0,1
Delaware,1,1,0,0,0,0,0,0,0,1,...,0,1,0,1,1,1,1,1,1,1
Dist. of Columbia,1,1,0,1,0,0,1,0,1,1,...,0,1,1,0,1,1,1,1,1,1
Florida,1,1,0,1,0,0,0,0,1,0,...,1,1,1,0,1,1,1,0,0,1


In [5]:
policy_topic = pd.read_csv("../data/policy_topic.csv")

In [6]:
policy_topic

,State / Jurisdiction,Policy Title,Topic Model
0,Alabama,Amends Alabama Code Title 16. Education § 16-4...,5.0
1,Alabama,Alabama Code Title 16. Education § 16-40A-2 (e...,5.0
2,Alaska,LAWS OF ALASKA,2.0
3,Alaska,An Act renaming portions of the Alaska Safe Ch...,2.0
4,Alaska,Alaska Statutes 2020 AS 14.30.352,2.0
...,...,...,...
152,Washington,RCW 28A.230.020 Common school curriculum.,10.0
153,Wisconsin,115.35 Health problems education program.,10.0
154,Wisconsin,118.019 Human growth and development instruct...,8.0
155,West Virginia,§18-2-9. Required courses of instruction.,10.0
